In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from autils import *
# plt.style.use('./deeplearning.mplstyle')

# import logging
# logging.getLogger("tensorflow").setLevel(logging.ERROR)
# tf.autograph.set_verbosity(0)
# np.set_printoptions(precision=2)

In [3]:
# load dataset
x = np.load("data/X.npy")
y = np.load("data/y.npy")
# Visualize dataset
print ('The shape of X is: ' + str(x.shape))
print ('The shape of y is: ' + str(y.shape))

The shape of X is: (5000, 400)
The shape of y is: (5000, 1)


In [4]:
# Get 60% of the dataset as the training set. Put the remaining 40% in temporary variables: x_ and y_.
x_train, x_, y_train, y_ = train_test_split(x, y, test_size=0.40, random_state=1)

# Split the 40% subset above into two: one half for cross validation and the other for the test set
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

# Delete temporary variables
del x_, y_

print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}\n")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")

the shape of the training set (input) is: (3000, 400)
the shape of the training set (target) is: (3000, 1)

the shape of the cross validation set (input) is: (1000, 400)
the shape of the cross validation set (target) is: (1000, 1)

the shape of the test set (input) is: (1000, 400)
the shape of the test set (target) is: (1000, 1)


In [13]:
# Initialize lists that will contain the errors for each model
nn_train_mses = []
nn_cv_mses = []

# Build the models
nn_models = build_models()

# Loop over the the models
for model in nn_models:
    
    # Setup the loss and optimizer
    model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    )

    print(f"Training {model.name}...")
    
    # Train the model
    model.fit(
        x_train, y_train,
        epochs=300,
        verbose=0
    )
    
    print("Done!\n")

    
    # Record the training MSEs
    yhat = model.predict(x_train)
    train_mse = mean_squared_error(y_train, yhat) / 2
    nn_train_mses.append(train_mse)
    
    # Record the cross validation MSEs 
    yhat = model.predict(x_cv)
    cv_mse = mean_squared_error(y_cv, yhat) / 2
    nn_cv_mses.append(cv_mse)

    
# print results
print("RESULTS:")
for model_num in range(len(nn_train_mses)):
    print(
        f"Model {model_num+1}: Training MSE: {nn_train_mses[model_num]:.2f}, " +
        f"CV MSE: {nn_cv_mses[model_num]:.2f}"
        )

Training model_1...
Done!

32/32 [==============================] - 0s 524us/step
Training model_2...
Done!

32/32 [==============================] - 0s 671us/step
Training model_3...
Done!

32/32 [==============================] - 0s 712us/step
RESULTS:
Model 1: Training MSE: 1.53, CV MSE: 2.02
Model 2: Training MSE: 4.09, CV MSE: 4.31
Model 3: Training MSE: 4.09, CV MSE: 4.31


In [14]:
# Select the model with the lowest CV MSE
model_num = 1

# Compute the test MSE
yhat = nn_models[model_num-1].predict(x_test)
test_mse = mean_squared_error(y_test, yhat) / 2
print(f"Test MSE: {test_mse:.2f}")

32/32 [==============================] - 0s 706us/step
Test MSE: 2.05
